In [14]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%autoreload 2

In [17]:
from mushroom.registration.bigwarp import (
    read_bigwarp_warp_field, register_multiplex, register_visium, register_he)
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

In [18]:
run_dir = '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/'
bigwarp_dir = os.path.join(run_dir, 'registration')
registered_dir = os.path.join(run_dir, 'registered')
Path(registered_dir).mkdir(parents=True, exist_ok=True)

In [19]:
metadata = yaml.safe_load(open(os.path.join(bigwarp_dir, 'metadata.yaml')))
metadata

{'HT112C1': {'data': {'visium': {'HT112C1-U1': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U1_ST_Bn1/outs',
    'HT112C1-U2': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U2_ST_Bn1/outs'}},
  'ids': ['s0', 's1'],
  'order': ['HT112C1-U1', 'HT112C1-U2'],
  'registered_pixels_per_micron': 2.480402478562179,
  'scale': 0.1},
 'HT206B1': {'data': {'visium': {'HT206B1-U2': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U2Z1B1/outs',
    'HT206B1-U3': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U3Z1B1/outs',
    'HT206B1-U4': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U4Z1B1/outs',
    'HT206B1-U5': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U5Z1B1/outs'}},
  'ids': ['s0', 's1', 's2', 's3'],
  'order': ['HT206B1-U2', 'HT206B1-U3', 'HT206B1-U4', 'HT206B1-U5'],
  

In [20]:
fps = sorted(utils.listfiles(bigwarp_dir, regex=r'ddf.tif$'))
fps

['/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT112C1/s1_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT206B1/s1_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT206B1/s2_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT206B1/s3_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT225C1/s1_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT225C1/s2_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT225C1/s3_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT225C1/s4_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT226C1/s1_ddf.tif',
 '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registration/HT235B1/s1_ddf.tif',


In [21]:
case_to_ddfs = {}
for fp in fps:
    case = fp.split('/')[-2]
    if case not in case_to_ddfs:
        case_to_ddfs[case] = {}
        
    scale = metadata[case]['scale']
    
    sid = re.findall(r'(s[0-9])_ddf.tif', fp)[0]
    ddf = read_bigwarp_warp_field(fp, scale)
    case_to_ddfs[case][sid] = ddf
    print(case, ddf.shape)
case_to_ddfs.keys()

HT112C1 torch.Size([2, 19880, 19630])
HT206B1 torch.Size([2, 19910, 19640])
HT206B1 torch.Size([2, 19910, 19640])
HT206B1 torch.Size([2, 19910, 19640])
HT225C1 torch.Size([2, 17120, 17180])
HT225C1 torch.Size([2, 17420, 17180])
HT225C1 torch.Size([2, 17420, 17180])
HT225C1 torch.Size([2, 17420, 17180])
HT226C1 torch.Size([2, 20940, 19640])
HT235B1 torch.Size([2, 20970, 19640])
HT243B1-H3 torch.Size([2, 19920, 19630])
HT243B1-H3 torch.Size([2, 19920, 19630])
HT243B1-H3 torch.Size([2, 19920, 19630])
HT243B1-H4 torch.Size([2, 19890, 19590])
HT268B1 torch.Size([2, 18880, 19640])
HT268B1 torch.Size([2, 18880, 19640])
HT268B1 torch.Size([2, 18880, 19640])
HT268B1 torch.Size([2, 18880, 19640])
HT271B1 torch.Size([2, 19880, 19640])
HT271B1 torch.Size([2, 19880, 19640])
HT271B1 torch.Size([2, 19880, 19640])
HT339B1 torch.Size([2, 19900, 19650])
HT397B1 torch.Size([2, 18640, 18370])
HT397B1 torch.Size([2, 18640, 18370])
HT397B1 torch.Size([2, 18640, 18370])
HT397B1 torch.Size([2, 18640, 18370])


dict_keys(['HT112C1', 'HT206B1', 'HT225C1', 'HT226C1', 'HT235B1', 'HT243B1-H3', 'HT243B1-H4', 'HT268B1', 'HT271B1', 'HT339B1', 'HT397B1', 'HT413C1', 'HT448C1'])

In [9]:
# dtype, case, sample, fp = 'visium', 'HT112C1', 'HT112C1-U2', '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U2_ST_Bn1/outs'
# data = metadata[case]
# target_pix_per_micron = data['registered_pixels_per_micron']
# ids = data['ids']
# order = data['order']

# ddf = case_to_ddfs[case].get(sid)
# adata = adata_from_visium(fp)
# adata, ddf.shape

In [10]:
# warped = register_visium(adata, ddf, target_pix_per_micron=target_pix_per_micron)

In [11]:
# sc.pl.spatial(adata, color='EPCAM')

In [12]:
# sc.pl.spatial(warped, color='EPCAM')

In [13]:
for case, data in metadata.items():
    ppm = data['registered_pixels_per_micron']
    ids = data['ids']
    order = data['order']
    directory = os.path.join(registered_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)

    for dtype, mapping in data['data'].items():
        for sample, fp in mapping.items():
            print(dtype, sample, fp)
            sid = ids[order.index(sample)]
            ddf = case_to_ddfs[case].get(sid)

            if dtype == 'he':
                he = tifffile.imread(fp)
                he = rearrange(he, 'h w c -> c h w') # channel dim must be first
                if ddf is not None:
                    he = register_he(he, ddf)
                tifffile.imwrite(
                    os.path.join(directory, f'{sid}_he.tif'),
                    data=rearrange(he, 'c h w -> h w c'),
                    resolution=(ppm, ppm),
                    compression='LZW',
                    metadata={
                        'axes': 'YXC',
                        'unit': 'um',
                    }
                )
            elif dtype == 'visium':
                adata = adata_from_visium(fp)
                if ddf is not None:
                    adata = register_visium(
                        adata, ddf, target_pix_per_micron=ppm
                    )
                adata.write_h5ad(os.path.join(directory, f'{sid}_visium.h5ad'))
            elif dtype == 'multiplex':
                channels, data = extract_ome_tiff(fp, as_dict=False)
                if ddf is not None:
                    data = register_multiplex(data, ddf)
                write_basic_ome_tiff(
                    os.path.join(directory, f'{sid}_multiplex.ome.tiff'),
                    data, channels, pix_per_micron=ppm
                )

visium HT112C1-U1 /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U1_ST_Bn1/outs


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


visium HT112C1-U2 /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U2_ST_Bn1/outs


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future versio

visium HT206B1-U2 /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U2Z1B1/outs


...

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future versio

visium HT448C1-U1 /diskmnt/Datasets/Spatial_Transcriptomics/outputs_FFPE/Human/HT448C1/Th1K1/HT448C1-Th1K1Fp1U1Bp1/outs


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


visium HT448C1-U13 /diskmnt/Datasets/Spatial_Transcriptomics/outputs_FFPE/Human/HT448C1/Th1K1/HT448C1-Th1K1Fp1U13Bp1/outs


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future versio

In [40]:
fps = sorted(utils.listfiles(registered_dir, regex=r'\/s[0-9]+.*$'))
case_to_id_to_fp = {}
for fp in fps:
    case = fp.split('/')[-2]
    sid = re.sub(r'(s[0-9]+)_.*$', r'\1', fp.split('/')[-1])
    if case not in case_to_id_to_fp:
        case_to_id_to_fp[case] = {}
    
    case_to_id_to_fp[case][sid] = fp
case_to_id_to_fp

{'HT112C1': {'s0': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s0_visium.h5ad',
  's1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s1_visium.h5ad'},
 'HT206B1': {'s0': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s0_visium.h5ad',
  's1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s1_visium.h5ad',
  's2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s2_visium.h5ad',
  's3': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s3_visium.h5ad'},
 'HT225C1': {'s0': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT225C1/s0_visium.h5ad',
  's1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT225C1/s1_visium.h5ad',
  's2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT225C1/s

In [41]:
registered_metadata = metadata.copy()
for case, data in registered_metadata.items():
    for dtype, mapping in data['data'].items():
        for sample, fp in mapping.items():
            sid = data['ids'][data['order'].index(sample)]
            mapping[sample] = case_to_id_to_fp[case][sid]
registered_metadata

{'HT112C1': {'data': {'visium': {'HT112C1-U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s0_visium.h5ad',
    'HT112C1-U2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s1_visium.h5ad'}},
  'ids': ['s0', 's1'],
  'order': ['HT112C1-U1', 'HT112C1-U2'],
  'registered_pixels_per_micron': 2.480402478562179,
  'scale': 0.1},
 'HT206B1': {'data': {'visium': {'HT206B1-U2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s0_visium.h5ad',
    'HT206B1-U3': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s1_visium.h5ad',
    'HT206B1-U4': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s2_visium.h5ad',
    'HT206B1-U5': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s3_visium.h5ad'}},
  'ids': ['s0', 's1', 's2', 's3'],
  'order': ['HT206B1-U2', 'HT206B1-U3', 'HT206B1-U4', '

In [42]:
yaml.safe_dump(registered_metadata, open(os.path.join(registered_dir, 'metadata.yaml'), 'w'))